In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def spin_init(L,alinged=False):
    if alinged==False:
        lattice= np.random.choice((-1,1),size=(L,L,L)).astype(np.int8)

    if alinged==True:
        lattice=np.ones((L,L,L),dtype=np.int8)
    return lattice

In [3]:
# calculation of magnetization of the lattice
def magnetization(lattice,L):
    return np.sum(lattice)

In [4]:
def PBC(ind,L):
    """applies to index+1, to get the first element when index==L-1"""
    if ind == L-1:
        return 0
    else:
        return ind+1

In [5]:
def energy_site(lattice,L,i,j,k):
    '''
    calculation of interaction energy of each spin and total lattice energy
    '''
    #print ('we selected: ',lattice[i,j,k])
    #print (lattice[i-1,j,k] , lattice[PBC(i,L),j,k] , lattice[i,j-1,k] , lattice[i,PBC(j,L),k] , lattice[i,j,k-1] , lattice[i,j,PBC(k,L)]
    return lattice[i,j,k]*(lattice[i-1,j,k]+lattice[PBC(i,L),j,k]+lattice[i,j-1,k]+lattice[i,PBC(j,L),k]+lattice[i,j,k-1]+lattice[i,j,PBC(k,L)])

In [6]:
def energy_total(lattice,L):
    sum=0
    for i in range(L):
        for j in range(L):
            for k in range(L):
                sum-=energy_site(lattice,L,i,j,k) # -ve sign due to -J (ferromagnetic) in the hamitonian
    
    return sum/2 # correct for overcounting due to overlapping

In [7]:
# determine if a spin is flipped according to monte-carlo rules
def flip(lattice,L,T):
    for i in range(L):
        for j in range(L):
            for k in range(L):
                if energy_site(lattice,L,i,j,k)<0:
                    lattice[i][j][k]=-lattice[i][j][k]
                else:
                    if np.e**(-2*energy_site(lattice,L,i,j,k)/T)>np.random.uniform(0,1):
                        lattice[i][j][k]=-lattice[i][j][k]

In [8]:
from random import randint
def temp_label(T):
    global sampleid
    
    X[sampleid] = (lattice+1)/2
    X_temp[sampleid] = T
    if T < Tc:
        X_label[sampleid] = 1
    if T > Tc:
        X_label[sampleid] = 0
    if T > Tc:
        X_label[sampleid] = randint(0,1)

    sampleid += 1
    #print(sampleid)

In [9]:
# the main calculation routine to calculate the values
def main(lattice,L,N_run,T):
    M=0
    m=0
    M_sq=0
    E=0
    e=0
    E_sq=0
    
    ### stepping only lattice; no data sampling #####
    for k in range (N_ss):
        flip(lattice,L,T)
    
    run=0    
    for k in range (N_ss,N_run):
        flip(lattice,L,T)
        m=np.sum(lattice) #magnetization(lattice,L)
        M+=m
        M_sq+=m**2
        e=energy_total(lattice,L)
        E+=e
        E_sq+=e**2

        #sampleid = count*(N_run - int(fracN_ss*N_run)) + run
        #print(sampleid,count)
        temp_label(T)
        run += 1
    
    M_mean=M/(N_run-N_ss)
    M_var=M_sq/(N_run-N_ss)-M_mean**2
    E_mean=E/(N_run-N_ss)
    E_var=E_sq/(N_run-N_ss)-E_mean**2
    C=E_var/T**2/L**3
    Chi=M_var/T/L**3

    return T,M_mean/L**3,E_mean/L**3,C,Chi # temporary take out lattice, which is global

In [18]:
## set your parameters here ##############
L=10;          ###=10  for trouble shooting; 50   for production run
N_run=10;     ###=100 for trouble shooting; 2000 for production run
fracN_ss=0.43   ### = 0.5 default. Must be a fractional number >=0 && < 1.
DeltaT=0.1     ###=1.0 for trouble shooting; 0.05 for production run
Tini=0.000;Tlast=5.0;  #default: Tini=0.0; Tlast=5.0
lattice=spin_init(L,alinged=False)
## end of set your parameters here ##############

In [19]:
### initialization ####
N_ss = int(fracN_ss*N_run);   ### step at which sampling of data begins
T_range = np.arange(Tini,Tlast+DeltaT,DeltaT)
sample_size = (N_run-N_ss)*T_range.shape[0]

###### data arrays ##########
ctn = np.zeros((T_range.shape[0],5),dtype=np.float32)
X = np.zeros((sample_size,L,L,L),dtype=np.int8)
X_label = np.zeros((sample_size),dtype=np.int8)
X_temp = np.zeros((sample_size),dtype=np.float32)
Tc = 4.5

# print informations
print(20*'=',' Sampling Summary','='*20)
print('Lattice shape = ',lattice.shape)
print('Temperature range = ',(Tini,Tlast),',at DeltaT = ',DeltaT)
print ('total run per temperature= ',N_run)
print ('runs before sampling = ',N_ss)
print ('sampling runs per temperature = ',N_run-int(fracN_ss*N_run))
print ('total number of temperature batchs = ',T_range.shape[0])
print ('total sample size = ',sample_size)
print(20*'=',' end of Summary ','='*20)
print('\n')

====================  Sampling Summary ====================
Lattice shape =  (10, 10, 10)
Temperature range =  (0.0, 5.0) ,at DeltaT =  0.1
total run per temperature=  10
runs before sampling =  4
sampling runs per temperature =  6
total number of temperature batchs =  51
total sample size =  306
====================  end of Summary  ====================




In [20]:
### begin calculation ##########
print(20*'=',' Sampling in progress ','='*20)

batch = 0
sampleid = 0
for T in T_range:
    print("T=",T)
    ctn[batch][0],ctn[batch][1],ctn[batch][2],ctn[batch][3],ctn[batch][4]=main(lattice,L,N_run,T)
    X[batch] = (lattice+1)/2
    batch +=1

print(20*'=',' end of Sampling ','='*20)
print('\n')

====================  Sampling in progress  ====================
T= 0.0


/share/apps/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/share/apps/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':
/share/apps/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in double_scalars
/share/apps/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars


T= 0.1
T= 0.2
T= 0.30000000000000004
T= 0.4
T= 0.5
T= 0.6000000000000001
T= 0.7000000000000001
T= 0.8
T= 0.9
T= 1.0
T= 1.1
T= 1.2000000000000002
T= 1.3
T= 1.4000000000000001
T= 1.5
T= 1.6
T= 1.7000000000000002
T= 1.8
T= 1.9000000000000001
T= 2.0
T= 2.1
T= 2.2
T= 2.3000000000000003
T= 2.4000000000000004
T= 2.5
T= 2.6
T= 2.7
T= 2.8000000000000003
T= 2.9000000000000004
T= 3.0
T= 3.1
T= 3.2
T= 3.3000000000000003
T= 3.4000000000000004
T= 3.5
T= 3.6
T= 3.7
T= 3.8000000000000003
T= 3.9000000000000004
T= 4.0
T= 4.1000000000000005
T= 4.2
T= 4.3
T= 4.4
T= 4.5
T= 4.6000000000000005
T= 4.7
T= 4.800000000000001
T= 4.9
T= 5.0
====================  end of Sampling  ====================




In [24]:
print(20*'=',' Building test datasets ','='*20)

np.savez('test_dataset.npz',x_train=X,x_temp=X_temp)
print('dataset saved : ', np.load('test_dataset.npz').files)

print(20*'=',' end of test dataset preparation ','='*20)
print('\n')

====================  Building test datasets  ====================
dataset saved :  ['x_train', 'x_temp']
====================  end of test dataset preparation  ====================


